In [0]:
%sql
USE pinterest_data_9105411ea84a;

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
# Reading data from Kinesis-Prod-Stream (partition key 9105411ea84a_pin') and deserialising the Data column

df_kinesis_pin = spark \
.readStream \
.format('kinesis') \
.option('streamName','Kinesis-Prod-Stream') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

df_kinesis_pin = df_kinesis_pin.filter(df_kinesis_pin.partitionKey == "9105411ea84a_pin")
df_kinesis_pin = df_kinesis_pin.selectExpr("CAST(data as STRING) jsonData")

display(df_kinesis_pin)

jsonData
"{""index"":7528,""unique_id"":""fbe53c66-3442-4773-b19e-d3ec6f54dddf"",""title"":""No Title Data Available"",""description"":""No description available Story format"",""poster_name"":""User Info Error"",""follower_count"":""User Info Error"",""tag_list"":""N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e"",""is_image_or_video"":""multi-video(story page format)"",""img_src"":null,""downloaded"":0,""save_location"":""Local save in /data/mens-fashion"",""category"":""mens-fashion""}"
"{""index"":2863,""unique_id"":""9bf39437-42a6-4f02-99a0-9a0383d8cd70"",""title"":""25 Super Fun Summer Crafts for Kids - Of Life and Lisa"",""description"":""Keep the kids busy this summer with these easy diy crafts and projects. Creative and…"",""poster_name"":""Of Life & Lisa | Lifestyle Blog"",""follower_count"":""124k"",""tag_list"":""Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls"",""is_image_or_video"":""image"",""img_src"":null,""downloaded"":1,""save_location"":""Local save in /data/diy-and-crafts"",""category"":""diy-and-crafts""}"
"{""index"":5730,""unique_id"":""1e1f0c8b-9fcf-460b-9154-c775827206eb"",""title"":""Island Oasis Coupon Organizer"",""description"":""Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the \""basic\"" couponer - holds up to 500 coupons with ease, and is made long enough so that you… "",""poster_name"":""Consuelo Aguirre"",""follower_count"":""0"",""tag_list"":""Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons"",""is_image_or_video"":""image"",""img_src"":null,""downloaded"":1,""save_location"":""Local save in /data/finance"",""category"":""finance""}"
"{""index"":8304,""unique_id"":""5b6d0913-25e4-43ab-839d-85d5516f78a4"",""title"":""The #1 Reason You’re Not His Priority Anymore - Matthew Coast"",""description"":""#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself"",""poster_name"":""Commitment Connection"",""follower_count"":""51k"",""tag_list"":""Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes"",""is_image_or_video"":""image"",""img_src"":null,""downloaded"":1,""save_location"":""Local save in /data/quotes"",""category"":""quotes""}"
"{""index"":8731,""unique_id"":""ea760f71-febf-4023-b592-d17396659039"",""title"":""20 Koi Fish Tattoos For Lucky Men"",""description"":""Koi fish tattoos are a popular choice for men who want to make a statement, thanks to their rich symbolism and bold design."",""poster_name"":""TheTrendSpotter"",""follower_count"":""211k"",""tag_list"":""Dr Tattoo,Wörter Tattoos,Pisces Tattoos,Tatoo Art,Dream Tattoos,Dope Tattoos,Mini Tattoos,Finger Tattoos,Body Art Tattoos"",""is_image_or_video"":""image"",""img_src"":null,""downloaded"":1,""save_location"":""Local save in /data/tattoos"",""category"":""tattoos""}"
"{""index"":1313,""unique_id"":""44662045-e891-4821-8a19-ebe7eedd371a"",""title"":""Liquid Lash Extensions Mascara"",""description"":""Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!"",""poster_name"":""Thrive Causemetics"",""follower_count"":""43k"",""tag_list"":""N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e"",""is_image_or_video"":""video"",""img_src"":null,""downloaded"":1,""save_location"":""Local save in /data/beauty"",""category"":""beauty""}"
"{""index"":4315,""unique_id"":""21b59ba9-829d-4c33-8c27-4cd4c56d26b8"",""title"":""Podcasts for Teachers or Parents of Teenagers"",""description"":""Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng… "",""poster_name"":""Math Giraffe"",""follower_count"":""25k"",""tag_list"":""Middle School Classroom,

In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, LongType, StringType, IntegerType, TimestampType, DoubleType

# Defining the schema for 9105411ea84a_pin
schema_pin = StructType([
    StructField("index", IntegerType(), True),
    StructField("unique_id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("poster_name", StringType(), True),
    StructField("follower_count", StringType(), True),
    StructField("tag_list", StringType(), True),
    StructField("is_image_or_video", StringType(), True),
    StructField("img_src", StringType(), True),
    StructField("downloaded", StringType(), True),
    StructField("save_location", StringType(), True),
    StructField("category", StringType(), True)
])

# Parse the jsonData column into a struct column using the defined schema
parsedDF = df_kinesis_pin.withColumn("parsedData", from_json(col("jsonData"), schema_pin))

# Flatten the parsed JSON to access individual fields
resultDF_pin = parsedDF.select("parsedData.*")

# Create a temporary view for cleaning purposes
resultDF_pin.createOrReplaceTempView("temp_view_pin")

display(resultDF_pin)


index,unique_id,title,description,poster_name,follower_count,tag_list,is_image_or_video,img_src,downloaded,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,User Info Error,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),null,0,Local save in /data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,Of Life & Lisa | Lifestyle Blog,124k,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,null,1,Local save in /data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",Consuelo Aguirre,0,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,null,1,Local save in /data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,Commitment Connection,51k,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,null,1,Local save in /data/quotes,quotes
8731,ea760f71-febf-4023-b592-d17396659039,20 Koi Fish Tattoos For Lucky Men,"Koi fish tattoos are a popular choice for men who want to make a statement, thanks to their rich symbolism and bold design.",TheTrendSpotter,211k,"Dr Tattoo,Wörter Tattoos,Pisces Tattoos,Tatoo Art,Dream Tattoos,Dope Tattoos,Mini Tattoos,Finger Tattoos,Body Art Tattoos",image,null,1,Local save in /data/tattoos,tattoos
1313,44662045-e891-4821-8a19-ebe7eedd371a,Liquid Lash Extensions Mascara,"Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!",Thrive Causemetics,43k,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",video,null,1,Local save in /data/beauty,beauty
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",Math Giraffe,25k,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,null,1,Local save in /data/education,education
10794,c4bd2577-a7bb-4409-bb7a-17d5ed7e1cf1,TireBuyer,Nissan GT-R. Sick.,Ray Uyemura,437,"Lowrider,Old Vintage Cars,Antique Cars,Austin Martin,Nissan Gtr Black,Jaguar,1959 Cadillac,Cadillac Ct6,Old School Cars",image,null,1,Local save in /data/vehicles,vehicles
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…","Living Low Key | Save Money, Make Money, & Frugal Living",26k,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",image,null,1,Local save in /data/finance,finance
5069,b75b6f87-deb3-444f-b29e-ce9161b2df49,The Vault: Curated & Refined Wedding Inspiration,Sacramento California Wedding 2 Chic Events & Design Jodi Yorston Photography Wilson Vineyards Barn Miosa Couture Yellow Barn Vineyard Outdoor Candles DIY,Style Me Pretty,6

In [0]:
# Reading data from Kinesis-Prod-Stream (partition key 9105411ea84a_geo') and deserialising the Data column

df_kinesis_geo = spark \
.readStream \
.format('kinesis') \
.option('streamName','Kinesis-Prod-Stream') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

df_kinesis_geo = df_kinesis_geo.filter(df_kinesis_geo.partitionKey == "9105411ea84a_geo")
df_kinesis_geo = df_kinesis_geo.selectExpr("CAST(data as STRING) jsonData")

display(df_kinesis_geo)

jsonData
"{""ind"":7528,""timestamp"":""2020-08-28T03:52:47"",""latitude"":-89.9787,""longitude"":-173.293,""country"":""Albania""}"
"{""ind"":2863,""timestamp"":""2020-04-27T13:34:16"",""latitude"":-5.34445,""longitude"":-177.924,""country"":""Armenia""}"
"{""ind"":5730,""timestamp"":""2021-04-19T17:37:03"",""latitude"":-77.015,""longitude"":-101.437,""country"":""Colombia""}"
"{""ind"":8304,""timestamp"":""2019-09-13T04:50:29"",""latitude"":-28.8852,""longitude"":-164.87,""country"":""French Guiana""}"
"{""ind"":8731,""timestamp"":""2020-07-17T04:39:09"",""latitude"":-83.104,""longitude"":-171.302,""country"":""Aruba""}"
"{""ind"":1313,""timestamp"":""2018-06-26T02:39:25"",""latitude"":77.0447,""longitude"":61.9119,""country"":""Maldives""}"
"{""ind"":4315,""timestamp"":""2019-12-15T03:51:28"",""latitude"":-45.8508,""longitude"":66.1003,""country"":""Cote d'Ivoire""}"
"{""ind"":10794,""timestamp"":""2022-01-01T02:26:50"",""latitude"":-89.5236,""longitude"":-154.567,""country"":""Cocos (Keeling) Islands""}"
"{""ind"":5494,""timestamp"":""2021-07-21T02:02:35"",""latitude"":-82.6768,""longitude"":-129.202,""country"":""Bulgaria""}"
"{""ind"":5069,""timestamp"":""2021-03-20T09:32:44"",""latitude"":-63.0063,""longitude"":-157.474,""country"":""Azerbaijan""}"


In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, IntegerType

# Define the schema for 9105411ea84a_geo
schema_geo = StructType([
    StructField("ind", IntegerType(), True),
    StructField("timestamp", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("country", StringType(), True)
])

# Parse the jsonData column into a struct column using the defined schema
parsedDF_geo = df_kinesis_geo.withColumn("parsedData", from_json(col("jsonData"), schema_geo))

# Flatten the parsed JSON to access individual fields
resultDF_geo = parsedDF_geo.select("parsedData.*")

# Create a temporary view for cleaning purposes
resultDF_geo.createOrReplaceTempView("temp_view_geo")

display(resultDF_geo)


ind,timestamp,latitude,longitude,country
7528,2020-08-28T03:52:47,-89.9787,-173.293,Albania
2863,2020-04-27T13:34:16,-5.34445,-177.924,Armenia
5730,2021-04-19T17:37:03,-77.015,-101.437,Colombia
8304,2019-09-13T04:50:29,-28.8852,-164.87,French Guiana
8731,2020-07-17T04:39:09,-83.104,-171.302,Aruba
1313,2018-06-26T02:39:25,77.0447,61.9119,Maldives
4315,2019-12-15T03:51:28,-45.8508,66.1003,Cote d'Ivoire
10794,2022-01-01T02:26:50,-89.5236,-154.567,Cocos (Keeling) Islands
5494,2021-07-21T02:02:35,-82.6768,-129.202,Bulgaria
5069,2021-03-20T09:32:44,-63.0063,-157.474,Azerbaijan


In [0]:
# Reading data from Kinesis-Prod-Stream (partition key 9105411ea84a_user') and deserialising the Data column

df_kinesis_user = spark \
.readStream \
.format('kinesis') \
.option('streamName','Kinesis-Prod-Stream') \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

df_kinesis_user = df_kinesis_user.filter(df_kinesis_user.partitionKey == "9105411ea84a_user")
df_kinesis_user = df_kinesis_user.selectExpr("CAST(data as STRING) jsonData")

display(df_kinesis_user)

jsonData
"{""ind"":7528,""first_name"":""Abigail"",""last_name"":""Ali"",""age"":20,""date_joined"":""2015-10-24T11:23:51""}"
"{""ind"":2863,""first_name"":""Dylan"",""last_name"":""Holmes"",""age"":32,""date_joined"":""2016-10-23T14:06:51""}"
"{""ind"":5730,""first_name"":""Rachel"",""last_name"":""Davis"",""age"":36,""date_joined"":""2015-12-08T20:02:43""}"
"{""ind"":8304,""first_name"":""Charles"",""last_name"":""Berry"",""age"":25,""date_joined"":""2015-12-28T04:21:39""}"
"{""ind"":8731,""first_name"":""Andrea"",""last_name"":""Alexander"",""age"":21,""date_joined"":""2015-11-10T09:27:42""}"
"{""ind"":1313,""first_name"":""Brittany"",""last_name"":""Jones"",""age"":32,""date_joined"":""2016-04-02T03:51:23""}"
"{""ind"":4315,""first_name"":""Michelle"",""last_name"":""Prince"",""age"":36,""date_joined"":""2015-12-20T16:38:13""}"
"{""ind"":10794,""first_name"":""Thomas"",""last_name"":""Turner"",""age"":34,""date_joined"":""2016-12-22T00:02:02""}"
"{""ind"":5494,""first_name"":""Anne"",""last_name"":""Allen"",""age"":27,""date_joined"":""2015-12-16T15:20:05""}"
"{""ind"":5069,""first_name"":""Amanda"",""last_name"":""Ball"",""age"":25,""date_joined"":""2016-01-13T17:36:30""}"


In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, LongType, StringType, IntegerType

# Define the schema for 9105411ea84a_user
schema_user = StructType([
    StructField("ind", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("date_joined", StringType(), True)
])

# Parse the jsonData column into a struct column using the defined schema
parsedDF_user = df_kinesis_user.withColumn("parsedData", from_json(col("jsonData"), schema_user))

# Flatten the parsed JSON to access individual fields
resultDF_user = parsedDF_user.select("parsedData.*")

# Create a temporary view for cleaning purposes
resultDF_user.createOrReplaceTempView("temp_view_user")

display(resultDF_user)


ind,first_name,last_name,age,date_joined
7528,Abigail,Ali,20,2015-10-24T11:23:51
2863,Dylan,Holmes,32,2016-10-23T14:06:51
5730,Rachel,Davis,36,2015-12-08T20:02:43
8304,Charles,Berry,25,2015-12-28T04:21:39
8731,Andrea,Alexander,21,2015-11-10T09:27:42
1313,Brittany,Jones,32,2016-04-02T03:51:23
4315,Michelle,Prince,36,2015-12-20T16:38:13
10794,Thomas,Turner,34,2016-12-22T00:02:02
5494,Anne,Allen,27,2015-12-16T15:20:05
5069,Amanda,Ball,25,2016-01-13T17:36:30


In [0]:
%sql

--Cleaning raw data

-- Cleaning data from 9105411ea84a_pin stream. 

CREATE OR REPLACE TEMPORARY VIEW clean_kinesis_pin AS
SELECT
  -- Renaming "index" to "ind"
  index AS ind,
  
  -- Cleaning and transforming string columns
  CASE WHEN TRIM(unique_id) = '' THEN NULL ELSE unique_id END AS unique_id,
  CASE WHEN TRIM(title) = '' THEN NULL ELSE title END AS title,
  CASE WHEN TRIM(description) = '' THEN NULL ELSE description END AS description,
  
  -- Cleaning follower_count
  CASE 
    WHEN TRIM(follower_count) = '' OR follower_count IS NULL THEN NULL
    WHEN TRIM(follower_count) RLIKE '^[0-9]+$' THEN CAST(follower_count AS INT)  -- Already a number
    WHEN TRIM(follower_count) LIKE '%M' THEN CAST(REPLACE(follower_count, 'M', '') AS DOUBLE) * 1000000
    WHEN TRIM(follower_count) LIKE '%k' THEN CAST(REPLACE(follower_count, 'k', '') AS DOUBLE) * 1000
    ELSE NULL
  END AS follower_count,
  
  CASE WHEN TRIM(poster_name) = '' THEN NULL ELSE poster_name END AS poster_name,
  CASE WHEN TRIM(tag_list) = '' THEN NULL ELSE tag_list END AS tag_list,
  CASE WHEN TRIM(is_image_or_video) = '' THEN NULL ELSE is_image_or_video END AS is_image_or_video,
  
  -- Treat empty values as NULL
  CASE WHEN TRIM(img_src) = '' THEN NULL ELSE img_src END AS img_src,
  
  -- Clean save_location by removing everything before the first "/"
  CASE 
    WHEN TRIM(save_location) = '' THEN NULL 
    ELSE regexp_extract(TRIM(save_location), '(/.*)$', 1)
  END AS save_location,
  
  CASE WHEN TRIM(category) = '' THEN NULL ELSE category END AS category

FROM temp_view_pin;



-- Cleaning data from 9105411ea84a_geo stream

CREATE OR REPLACE TEMPORARY VIEW clean_kinesis_geo AS
SELECT
  ind,
  country,
  -- Create a new column 'coordinates' by concatenating latitude and longitude with a comma separator.
  CONCAT(CAST(latitude AS STRING), ',', CAST(longitude AS STRING)) AS coordinates,

  -- Convert timestamp to TIMESTAMP (if it isn't already)
  CAST(timestamp AS TIMESTAMP) AS timestamp
FROM temp_view_geo;


-- Cleaning data from 9105411ea84a_user stream

CREATE OR REPLACE TEMPORARY VIEW clean_kinesis_user AS
SELECT
  ind,
  -- Create a new column 'user_name' by concatenating first_name and last_name with a space inbetween
  CONCAT(first_name, ' ', last_name) AS user_name,
  age,
  CAST(date_joined AS TIMESTAMP) AS date_joined
FROM temp_view_user;

In [0]:
%sql
SELECT * FROM clean_kinesis_pin;

ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,img_src,save_location,category
7528,fbe53c66-3442-4773-b19e-d3ec6f54dddf,No Title Data Available,No description available Story format,null,User Info Error,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",multi-video(story page format),null,/data/mens-fashion,mens-fashion
2863,9bf39437-42a6-4f02-99a0-9a0383d8cd70,25 Super Fun Summer Crafts for Kids - Of Life and Lisa,Keep the kids busy this summer with these easy diy crafts and projects. Creative and…,124000.0,Of Life & Lisa | Lifestyle Blog,"Summer Crafts For Kids,Fun Crafts For Kids,Summer Kids,Toddler Crafts,Crafts To Do,Diy For Kids,Summer Snow,Diys For Summer,Craft Ideas For Girls",image,null,/data/diy-and-crafts,diy-and-crafts
5730,1e1f0c8b-9fcf-460b-9154-c775827206eb,Island Oasis Coupon Organizer,"Description Coupon Organizer in a fun colorful fabric -island oasis, Great Size for the ""basic"" couponer - holds up to 500 coupons with ease, and is made long enough so that you…",0.0,Consuelo Aguirre,"Grocery Items,Grocery Coupons,Care Organization,Coupon Organization,Extreme Couponing,Couponing 101,Life Binder,Save My Money,Love Coupons",image,null,/data/finance,finance
8304,5b6d0913-25e4-43ab-839d-85d5516f78a4,The #1 Reason You’re Not His Priority Anymore - Matthew Coast,#lovequotes #matchmaker #matchmadeinheaven #loveyourself #respectyourself,51000.0,Commitment Connection,"Wise Quotes,Quotable Quotes,Words Quotes,Wise Words,Quotes To Live By,Great Quotes,Motivational Quotes,Inspirational Quotes,Funny Quotes",image,null,/data/quotes,quotes
8731,ea760f71-febf-4023-b592-d17396659039,20 Koi Fish Tattoos For Lucky Men,"Koi fish tattoos are a popular choice for men who want to make a statement, thanks to their rich symbolism and bold design.",211000.0,TheTrendSpotter,"Dr Tattoo,Wörter Tattoos,Pisces Tattoos,Tatoo Art,Dream Tattoos,Dope Tattoos,Mini Tattoos,Finger Tattoos,Body Art Tattoos",image,null,/data/tattoos,tattoos
1313,44662045-e891-4821-8a19-ebe7eedd371a,Liquid Lash Extensions Mascara,"Instantly create the look of lash extensions with this award-winning, best-selling mascara that won't clump, flake or smudge. Available in 3 shades!",43000.0,Thrive Causemetics,"N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e",video,null,/data/beauty,beauty
4315,21b59ba9-829d-4c33-8c27-4cd4c56d26b8,Podcasts for Teachers or Parents of Teenagers,"Podcasts for Teachers or Parents of Teenagers: Teaching teens middle school and high school can feel joyful and rewarding most days, but can also frustrate you with one challeng…",25000.0,Math Giraffe,"Middle School Classroom,High School Students,High School Teachers,Middle School Tips,High School Counseling,Ela Classroom,High School Science,Future Classroom,Google Classroom",image,null,/data/education,education
10794,c4bd2577-a7bb-4409-bb7a-17d5ed7e1cf1,TireBuyer,Nissan GT-R. Sick.,437.0,Ray Uyemura,"Lowrider,Old Vintage Cars,Antique Cars,Austin Martin,Nissan Gtr Black,Jaguar,1959 Cadillac,Cadillac Ct6,Old School Cars",image,null,/data/vehicles,vehicles
5494,8fb2af68-543b-4639-8119-de33d28706ed,Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",26000.0,"Living Low Key | Save Money, Make Money, & Frugal Living","Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",image,null,/data/finance,finance
5069,b75b6f87-deb3-444f-b29e-ce9161b2df49,The Vault: Curated & Refined Wedding Inspiration,Sacramento California Wedding 2 Chic Events & Design Jodi Yorston Photography Wilson Vineyards Barn Miosa Couture Yellow Barn Vineyard Outdoor Candles DIY,6000000.0,Style Me Pretty,"60th Anniversary Parties,Anniversary Decorations,Golden Anniversary,25th Wedding Anniversary,Anniversary Pictures,Anniversary Idea

In [0]:
%sql
SELECT * FROM clean_kinesis_geo;

ind,country,coordinates,timestamp
7528,Albania,"-89.9787,-173.293",2020-08-28T03:52:47Z
2863,Armenia,"-5.34445,-177.924",2020-04-27T13:34:16Z
5730,Colombia,"-77.015,-101.437",2021-04-19T17:37:03Z
8304,French Guiana,"-28.8852,-164.87",2019-09-13T04:50:29Z
8731,Aruba,"-83.104,-171.302",2020-07-17T04:39:09Z
1313,Maldives,"77.0447,61.9119",2018-06-26T02:39:25Z
4315,Cote d'Ivoire,"-45.8508,66.1003",2019-12-15T03:51:28Z
10794,Cocos (Keeling) Islands,"-89.5236,-154.567",2022-01-01T02:26:50Z
5494,Bulgaria,"-82.6768,-129.202",2021-07-21T02:02:35Z
5069,Azerbaijan,"-63.0063,-157.474",2021-03-20T09:32:44Z


In [0]:
%sql
SELECT * FROM clean_kinesis_user;

ind,user_name,age,date_joined
7528,Abigail Ali,20,2015-10-24T11:23:51Z
2863,Dylan Holmes,32,2016-10-23T14:06:51Z
5730,Rachel Davis,36,2015-12-08T20:02:43Z
8304,Charles Berry,25,2015-12-28T04:21:39Z
8731,Andrea Alexander,21,2015-11-10T09:27:42Z
1313,Brittany Jones,32,2016-04-02T03:51:23Z
4315,Michelle Prince,36,2015-12-20T16:38:13Z
10794,Thomas Turner,34,2016-12-22T00:02:02Z
5494,Anne Allen,27,2015-12-16T15:20:05Z
5069,Amanda Ball,25,2016-01-13T17:36:30Z


In [0]:
# Reading streaming DataFrames from temporary views
stream_pin = spark.sql("SELECT * FROM clean_kinesis_pin")
stream_geo = spark.sql("SELECT * FROM clean_kinesis_geo")
stream_user = spark.sql("SELECT * FROM clean_kinesis_user")

# Write the streaming DataFrame for pin data to the corresponding Delta table
query_pin = stream_pin.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/checkpoints/9105411ea84a_pin_table") \
    .table("9105411ea84a_pin_table")

# Write the streaming DataFrame for geo data to the corresponding Delta table
query_geo = stream_geo.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/checkpoints/9105411ea84a_geo_table") \
    .table("9105411ea84a_geo_table")

# Write the streaming DataFrame for user data to the corresponding Delta table
query_user = stream_user.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/checkpoints/9105411ea84a_user_table") \
    .table("9105411ea84a_user_table")
